<a href="https://colab.research.google.com/github/rjhanjee/my-first-repo/blob/main/testfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# gdp_app.py

import streamlit as st
import pandas as pd
import plotly.express as px
import requests

WIKI_URL = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'

@st.cache_data
def load_data():
    """
    Webscrapes the Wikipedia page for GDP (nominal) and performs data cleaning.
    """
    try:
        tables = pd.read_html(WIKI_URL)
    except (requests.exceptions.RequestException, ValueError) as e:
        st.error(f"Error fetching data: {e}")
        return None, None, None

    # Find the correct table indices based on heading structure
    imf_df = tables[2].copy()
    un_df = tables[3].copy()
    wb_df = tables[4].copy()

    # Data Cleaning Function
    def clean_gdp_df(df, column_name):
        df.columns = ['Country', 'Region', 'Estimate', 'Year']
        df = df[df['Country'].notna()]
        df['Estimate'] = pd.to_numeric(
            df['Estimate'].str.replace(',', '').str.strip(),
            errors='coerce'
        )
        df.dropna(subset=['Estimate'], inplace=True)
        return df.sort_values('Estimate', ascending=False)

    imf_df = clean_gdp_df(imf_df.iloc[2:], 'IMF Estimate')
    un_df = clean_gdp_df(un_df.iloc[1:], 'UN Estimate')
    wb_df = clean_gdp_df(wb_df.iloc[1:], 'World Bank Estimate')

    return imf_df, un_df, wb_df

def create_stacked_bar_chart(df, data_source):
    """
    Creates an interactive stacked bar chart using Plotly.
    """
    # Sort regions by total GDP for better visual hierarchy
    region_order = df.groupby('Region')['Estimate'].sum().sort_values(ascending=False).index

    fig = px.bar(
        df,
        x='Estimate',
        y='Region',
        color='Country',
        orientation='h',
        title=f'GDP (Nominal) by Region and Country ({data_source})',
        labels={'Estimate': 'GDP (US$ millions)', 'Country': 'Country'},
        hover_data={'Country': True, 'Estimate': ':.2s'}
    )

    fig.update_layout(
        barmode='stack',
        xaxis_title='GDP (US$ millions)',
        yaxis_title='Region',
        yaxis={'categoryorder': 'array', 'categoryarray': list(region_order)}
    )

    return fig

def main():
    st.set_page_config(layout="wide")
    st.title('Global GDP (Nominal) Dashboard')

    imf_df, un_df, wb_df = load_data()

    if imf_df is None or un_df is None or wb_df is None:
        st.error("Failed to load data. Please check the source URL.")
        return

    data_source_option = st.selectbox(
        'Select data source:',
        ('IMF', 'UN', 'World Bank')
    )

    if data_source_option == 'IMF':
        df = imf_df
    elif data_source_option == 'UN':
        df = un_df
    else:
        df = wb_df

    st.plotly_chart(create_stacked_bar_chart(df, data_source_option), use_container_width=True)

if __name__ == '__main__':
    main()

2025-09-03 01:28:33.178 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-03 01:28:33.181 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 01:28:33.184 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 01:28:33.327 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-03 01:28:33.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 01:28:33.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 01:28:33.330 No runtime found, using MemoryCacheStorageManager
2025-09-03 01:28:33.331 Thread 'Main

HTTPError: HTTP Error 403: Forbidden

In [2]:
%pip install streamlit plotly requests pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.9 MB/s eta 0:00:00


Once the libraries are installed, you can run the Streamlit app using the following command:

In [ ]:
!streamlit run gdp_app.py & npx localtunnel --port 8501

This will start the Streamlit server and create a public URL that you can click on to view your app.